### Индивидуальное задание
Вычислить значение функции $f$ с точностью до сотых в точках из списка $X$.
#### Указание. 
Воспользуйтесь методом evalf() для получения приближенного значения. Значения $f$ в точках из списка $X$ выведите в виде списка.

Устанавливаем numpy, sympy

In [598]:
!python -m pip install sympy --upgrade
!python -m pip install numpy --upgrade

Импортируем нужные библиотеки

In [599]:
import numpy as np
import sympy as sp
from sympy.abc import x
from sympy.polys.specialpolys import random_poly
import operator

Используемые функции (`EX`): 
- `ln(x)`
- `exp(x)`
- `sin(x)`
- `cos(x)`
- `tan(x)`

Используемые внутренние функции (`INNER_EX`) - полиномы 2-ой или 3-ей степени с целочисленными коэффициентами 

Итоговая функция строится следующим образом:  
`f(x) = {+, -}ex1(inner_ex1){+, -, *, /}{+, -}ex2(inner_ex2)`, где:
- `ex1`, `ex2` in `EX`; `ex1 != ex2`
- `inner_ex1`, `inner_ex2` in `INNER_EX`

In [600]:
EX = [sp.ln, sp.exp, sp.sin, sp.cos, sp.tan]
ARITHMETIC_OPS = [operator.add, operator.sub, operator.mul, operator.truediv]

In [601]:
def generate_inner_ex():
    return random_poly(x, np.random.randint(2, 4), -10, 10)

In [602]:
def generate_final_function():
    ex1, ex2 = list(map(lambda ex: np.random.choice([-1, 1]) * ex(generate_inner_ex()), np.random.choice(EX, 2, replace=False)))
    return sp.sympify(np.random.choice(ARITHMETIC_OPS)(ex1, ex2))

In [603]:
# Restricting potential point values to not get crazy numbers.
POINT_MIN = -1e8
POINT_MAX = 1e8
POINT_EPSILON = 1e-2

In [604]:
"""
Safely rounds the expression to n decimal points
(see: https://github.com/sympy/sympy/issues/21513).
"""
def safe_round(ex: sp.Expr, n):
    return round(eval(f'{ex.evalf()}'), n)

In [611]:
"""
Attempts to generate a random list of 3 points, lying within the domain of the given function.
On failure, returns an empty list. 
"""
def generate_points(f: sp.Expr):
    def test(p):
        res = f.subs(x, p) 
        return res.is_finite and res.is_real and POINT_MIN <= res <= POINT_MAX and abs(safe_round(res, 2)) >= POINT_EPSILON

    points = []
    
    # First point is always an integer in [-10, 10].
    first_points = [p for p in range(-10, 11)]
    np.random.shuffle(first_points)

    for p in first_points:
        if test(p):
            points.append(p)
            break
    else:
        return []
    
    # 2 other points are of form: a + b*PI, b != 0
    for point_count in range(2):
        for attempt in range(100):
            p = np.random.randint(-10, 11) + np.random.choice([-1, 1]) * np.random.randint(1, 11) * sp.pi
            if test(p) and p not in points:
                points.append(p)
                break
        else:
            return []
        
    return points

In [612]:
"""
Returns a pair of a function and 3 points, lying within the domain of it.
"""
def generate_final_function_and_points():
    while True:
        f = generate_final_function()
        ps = generate_points(f)
        if len(ps) > 0:
            return f, ps

In [613]:
def solve(f: sp.Expr, points):
    return [safe_round(f.subs(x, p), 2) for p in points]

In [608]:
HEADER = """\\documentclass[11pt]{report}
\\usepackage[T2A]{fontenc}
\\usepackage[utf8]{inputenc}
\\usepackage[russian]{babel}
\\usepackage{amsmath,amssymb}
\\usepackage{graphicx}
\\oddsidemargin=-19mm
\\topmargin=-30mm
\\textheight 26cm
\\hsize 18cm
\\textwidth 20cm
\\begin{document}

\\pagestyle{empty}
{\\bf Индивидуальное задание.}
Вычислить значение функции $f$ с точностью до сотых в точках из списка $X$.
Указание. Воспользуйтесь методом evalf() для получения приближенного значения. Значения $f$ в точках из списка $X$ выведите в виде списка.\n\n"""

TAIL = "\n\\end{document}"
VARIANT_COUNT = 150

In [615]:
def generate_tasks_and_solutions(tasks_filepath: str, solutions_filepath: str):
    def head(variant_n: int):
        return r"""\begin{center}
    \noindent\rule{8cm}{0.4pt}
\end{center}
Вариант \(""" + str(variant_n) + "\\)\n"
    
    def desc(f: sp.Expr, ps, solution: bool):
        return r"""\begin{align*}
    f(x) = """ + sp.latex(f) + f" && {'Solution' if solution else 'X'} = " + sp.latex(ps) + r"""
\end{align*}
"""

    with open(tasks_filepath, "w+") as f_tasks, open(solutions_filepath, "w+") as f_solutions:
        f_tasks.write(HEADER)
        f_solutions.write(HEADER)

        for variant in range(VARIANT_COUNT):
            print(f"Generating variant {variant}...")
            f, ps = generate_final_function_and_points()
            head_latex = head(variant)

            f_tasks.write(head_latex)
            f_solutions.write(head_latex)
            
            f_tasks.write(desc(f, ps, False))
            f_solutions.write(desc(f, solve(f, ps), True))

        f_tasks.write(TAIL)
        f_solutions.write(TAIL)

In [616]:
generate_tasks_and_solutions("../out/task-1.tex", "../out/task-ans-1.tex")

Generating variant 0...
Generating variant 1...
Generating variant 2...
Generating variant 3...
Generating variant 4...
Generating variant 5...
Generating variant 6...
Generating variant 7...
Generating variant 8...
Generating variant 9...
Generating variant 10...
Generating variant 11...
Generating variant 12...
Generating variant 13...
Generating variant 14...
Generating variant 15...
Generating variant 16...
Generating variant 17...
Generating variant 18...
Generating variant 19...
Generating variant 20...
Generating variant 21...
Generating variant 22...
Generating variant 23...
Generating variant 24...
Generating variant 25...
Generating variant 26...
Generating variant 27...
Generating variant 28...
Generating variant 29...
Generating variant 30...
Generating variant 31...
Generating variant 32...
Generating variant 33...
Generating variant 34...
Generating variant 35...
Generating variant 36...
Generating variant 37...
Generating variant 38...
Generating variant 39...
Generating